<a href="https://colab.research.google.com/github/GheorgheBriana/Big-data-faculty/blob/main/lab2_part1_bigdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from time import time #time() ne va ajuta sa returnam timpul curent in secunde
from pyspark import SparkContext #pentru a utiliza spark

In [22]:
from pyspark import SparkContext
from time import time

for j in range(1, 5):  # loop de la 1 la 4 inclusiv; j reprezintă numărul de thread-uri
    sc = SparkContext(master="local[%d]" % (j))  # creează un nou SparkContext. Spark va rula local pe j threads
    print(sc.defaultParallelism)  # afișează valoarea lui j
    t0 = time()  # salvează timpul curent pentru a măsura cât durează execuția următoarelor operații

    for i in range(10):  # rulăm de mai multe ori pentru a măsura timpul mai exact
        sc.parallelize([1, 2] * 10000).reduce(lambda x, y: x + y)
        # vom avea 20.000 de elemente (2 elemente * 10.000) pentru a simula un volum mare de date
        # => vom avea o listă de tipul: [1, 2, 1, 2, 1, 2, ..., 1, 2] (de 10.000 de ori)
        # lista va fi transformată într-un RDD (pentru a o împărți în mai multe thread-uri)
        # reduce(lambda x, y: x + y) => adună toate cele 20.000 de elemente din RDD

    print(f"{j} executors, time = {time() - t0}")
    # inserăm valoarea lui j în text
    # {time() - t0} - calculează și inserează cât timp a trecut de la începutul măsurătorii
    # t0 = time(): înainte să înceapă procesarea; time() timpul curent
    sc.stop()  # Oprește SparkContext-ul la sfârșit


1
1 executors, time = 2.7330596446990967
2
2 executors, time = 3.2099251747131348
3
3 executors, time = 5.611538410186768
4
4 executors, time = 5.551335573196411


In [23]:
cat/proc/cpuinfo
#afiseaza informatii despre procesor

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0xffffffff
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa mmio_stale_data retbleed bhi
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
ad

In [30]:
#Creare RDD cu un milion de elemente
sc = SparkContext(master="local[2]")

In [33]:
%%time
#masoare timpul de executie al intregii celule
rdd1 = sc.parallelize(range(1000000)) #definire RDD cu un milion elemente

CPU times: user 2.17 ms, sys: 0 ns, total: 2.17 ms
Wall time: 5.04 ms


In [34]:
from math import cos #improtam functia cos din modulul math

def taketime(x):
  [cos(j) for j in range(100)] #calculeaza cosinului pentru numerele [0, 100)
  return cos(x) #returnare cosinus de x

In [35]:
%%time
taketime(2)

#CPU times: timpul efectiv in care procesorul a fost utilizat pentru a
  # efectua calculele
#Wall time: timpul total care a trecut pe ceasul real, adica timpul in care
  # intregul proces a fost activ
# daca cele doua au valori apropiate => proces relativ si eficient
# altfel => procesul a fost blocat sau a asteptat ar

CPU times: user 70 µs, sys: 0 ns, total: 70 µs
Wall time: 76.1 µs


-0.4161468365471424

In [36]:
%%time
interim = rdd1.map(lambda x: taketime(x))
#map va lua fiecare element din rdd1 (RDD-ul cu un milion de elemente) si v-om calcula cosinusul fiecarei valori
# cand se aplica functia map => Spark nu calculează imediat rezultatul.
# Acesta construieste un plan de executie care va spune ce sa faca atunci cand va veni momentul sa calculeze rezultatele
# acest lucru se datoreaza evaluarii lazy

CPU times: user 93 µs, sys: 0 ns, total: 93 µs
Wall time: 97.8 µs


In [41]:
print(interim.toDebugString().decode())
# afiseaza structura interna a RDD-ului interim
# toDebugString() afișează detalii despre cum sunt distribuite datele în RDD
# decode(): folosit pentru a decodifica string-ul returna de toDebugString() intr-un format citibil


(2) PythonRDD[2] at RDD at PythonRDD.scala:53 []
 |  ParallelCollectionRDD[1] at readRDDFromFile at PythonRDD.scala:289 []


In [42]:
%%time
print('output =', interim.reduce(lambda x, y: x+y))
#lambda x, y: x+y: aduna toate elementele din RDD
#reduce(): incepe cu doua elemente, le combina, apoi combina rezultatele pana cand se ajunge la un singur rezultat final

output = -0.2887054679684353
CPU times: user 49.2 ms, sys: 2.93 ms, total: 52.1 ms
Wall time: 10.7 s


In [43]:
1000000*31e-6

31.0

In [44]:
%%time
print(interim.filter(lambda x:x>0).count())
# filter: filtreaza RDD-ul pe baza unei conditii => se filtreaza elementele mai mari ca 0
# count(): aduna numarul de elemente
# => functia afiseaza numarul de elemente mai mari ca 0

500000
CPU times: user 46.6 ms, sys: 5.68 ms, total: 52.3 ms
Wall time: 10.1 s


In [48]:
#cache -> metoda care stocheaza temporar datele unui RDD in memoria RAM
# Utila pentru a accesa aceleasi date de mai multe ori intr-un loc rapid + evitarea recalcularii lor
# prin cache, spark salveaza datele in memorie si le va folosi cand sunt necesare, fara a recalcula

#fara cache:
interim = rdd1.map(lambda x: taketime(x))

In [49]:
#adagare cache:
interim = rdd1.map(lambda x: taketime(x)).cache()

In [50]:
#Afisare graf de dependente pentru iterim
print(interim.toDebugString().decode())

(2) PythonRDD[5] at RDD at PythonRDD.scala:53 [Memory Serialized 1x Replicated]
 |  ParallelCollectionRDD[1] at readRDDFromFile at PythonRDD.scala:289 [Memory Serialized 1x Replicated]


In [64]:
%%time
#Obțineți suma elementelor cu ajutorul metodei reduce și a cache-ului
interim.cache()
result = interim.reduce(lambda x, y: x+y)
print(result)

-0.2887054679684353
CPU times: user 6.02 ms, sys: 2.82 ms, total: 8.84 ms
Wall time: 594 ms


In [65]:
%%time
result2 = interim.filter(lambda x: x>0).count()
print(result2)

500000
CPU times: user 5.01 ms, sys: 2.82 ms, total: 7.82 ms
Wall time: 546 ms
